In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor


patient_data = pd.read_csv('dataset.csv')

In [4]:
patient_data

,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,Unnamed: 83,hospital_death
0,66154,25312,118,68.0,22.730000,0,Caucasian,M,180.3,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
1,114252,59342,81,77.0,27.420000,0,Caucasian,F,160.0,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0
2,119783,50777,118,25.0,31.950000,0,Caucasian,F,172.7,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
3,79267,46918,118,81.0,22.640000,1,Caucasian,F,165.1,Operating Room / Recovery,...,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,NaN,0
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,91592,78108,30,75.0,23.060250,0,Caucasian,M,177.8,Floor,...,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular,NaN,0
91709,66119,13486,121,56.0,47.179671,0,Caucasian,F,183.0,Floor,...,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
91710,8981,58179,195,48.0,27.236914,0,Caucasian,M,170.2,Accident & Emergency,...,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
91711,33776,120598,66,NaN,23.297481,0,Caucasian,F,154.9,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0


In [5]:
# These are all the columns in the data set
for col in patient_data.columns:
    print(col)

encounter_id
patient_id
hospital_id
age
bmi
elective_surgery
ethnicity
gender
height
icu_admit_source
icu_id
icu_stay_type
icu_type
pre_icu_los_days
weight
apache_2_diagnosis
apache_3j_diagnosis
apache_post_operative
arf_apache
gcs_eyes_apache
gcs_motor_apache
gcs_unable_apache
gcs_verbal_apache
heart_rate_apache
intubated_apache
map_apache
resprate_apache
temp_apache
ventilated_apache
d1_diasbp_max
d1_diasbp_min
d1_diasbp_noninvasive_max
d1_diasbp_noninvasive_min
d1_heartrate_max
d1_heartrate_min
d1_mbp_max
d1_mbp_min
d1_mbp_noninvasive_max
d1_mbp_noninvasive_min
d1_resprate_max
d1_resprate_min
d1_spo2_max
d1_spo2_min
d1_sysbp_max
d1_sysbp_min
d1_sysbp_noninvasive_max
d1_sysbp_noninvasive_min
d1_temp_max
d1_temp_min
h1_diasbp_max
h1_diasbp_min
h1_diasbp_noninvasive_max
h1_diasbp_noninvasive_min
h1_heartrate_max
h1_heartrate_min
h1_mbp_max
h1_mbp_min
h1_mbp_noninvasive_max
h1_mbp_noninvasive_min
h1_resprate_max
h1_resprate_min
h1_spo2_max
h1_spo2_min
h1_sysbp_max
h1_sysbp_min
h1_sysbp_

In [9]:
print(len(patient_data.columns))

85


In [21]:
features = ['age', 'bmi', 'gender', 'ethnicity', 'elective_surgery','weight',
            'icu_admit_source', 'icu_type', 'pre_icu_los_days','arf_apache',
            'gcs_eyes_apache', 'gcs_motor_apache', 'gcs_unable_apache', 'gcs_verbal_apache',
            'heart_rate_apache', 'intubated_apache', 'map_apache', 'resprate_apache',
            'temp_apache', 'ventilated_apache','d1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max',
            'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min',
            'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'h1_diasbp_max',
            'h1_diasbp_min', 'h1_heartrate_max', 'h1_heartrate_min', 'h1_mbp_max', 'h1_mbp_min',
            'h1_resprate_max', 'h1_resprate_min', 'h1_sysbp_max', 'h1_sysbp_min', 'd1_glucose_max',
            'd1_glucose_min', 'd1_potassium_max', 'd1_potassium_min', 'aids', 'cirrhosis', 
            'diabetes_mellitus', 'hepatic_failure', 'immunosuppression', 'leukemia', 'lymphoma',
            'solid_tumor_with_metastasis']

In [27]:
# the fillna part helps the model recognize when information is missing
X = pd.get_dummies(patient_data[features]).fillna(-1)
y = patient_data['hospital_death']

In [28]:
# This is the proper way to make a validation set with equal distributions as your training set
from sklearn.model_selection import train_test_split
# Split data into training and validation data , for both features and target
# The split is based on a random number generator. Supplying  numeric value to
# the random_state argument guarantees we get the same split every single time we
# run this script
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0, test_size = 0.1)

#y is synonymous with the targets

#Define Model
survival_model = DecisionTreeRegressor()

#fit model
survival_model.fit(train_X, train_y)

#Get predictions
val_predictions = survival_model.predict(val_X)




In [29]:
val_predictions

array([1., 1., 0., ..., 0., 0., 0.])

In [49]:
# Turns val_y into an array which is used in code below to calculate accuracy of model
validation_arr = pd.DataFrame(val_y).to_numpy()

In [50]:
validation_arr

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [33]:
print(len(train_X))
print(len(val_X))
print(len(train_y))
print(len(val_y))
print(len(val_predictions))

82541
9172
82541
9172
9172


In [51]:
a = 0
for i in range(0,9172):
    if val_predictions[i] == validation_arr[i]:
        a += 1
        
print(a)
    

7998


In [54]:
#accuracy of my model
print('The accuracy of my model is:', a/9172 * 100, '%')

The accuracy of my model is: 87.20017444395988 %
